# fireflies - Extended Feature Selection
* StelllarAlgo Data Science
* Ryan Kazmerik & Nakisa Rad
* Feb 22, 2022

## Hypothesis
Two very important components of a machine learning model are feature selection and feature engineering. Our idea is that adding some more features to the StellarAlgo retention model could improve performance of the model. This notebook will test the standard StellarAlgo retention model features.

## Experiment
This section details our experiment including querying data, data transformations, feature selection and modelling.

In [1]:
import getpass
import pyodbc
import pandas as pd
import warnings
import matplotlib.pyplot as plt

from pycaret.classification import *

warnings.filterwarnings('ignore')

### Let's connect to MSSQL and run a stored proc to get our dataset:

In [2]:
# connect to SQL Server.
SERVER = '34.206.73.189' 
DATABASE = 'datascience' 
USERNAME = 'nrad' 
PASSWORD = getpass.getpass(prompt='Enter your password')
CNXN = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USERNAME+';PWD='+ PASSWORD)

In [3]:
lkupclientid = 28 # fireflies

cursor = CNXN.cursor()

storedProc = (
    f"""Exec [stlrMILB].[ds].[getRetentionScoringModelData] {lkupclientid}"""
)

df = pd.read_sql(storedProc, CNXN)

# apply some data transformations
df["year"] = pd.to_numeric(df["year"])

CNXN.commit()
cursor.close()

df.head()
#df.info()

,lkupClientId,dimCustomerMasterId,customerNumber,year,productGrouping,totalSpent,recentDate,attendancePercent,renewedBeforeDays,isBuyer,source_tenure,tenure,distToVenue,totalGames,recency,missed_games_1,missed_games_2,missed_games_over_2,click_link,fill_out_form,open_email,send_email,unsubscribe_email,openToSendRatio,clickToSendRatio,clickToOpenRatio,posting_records,resale_records,resale_atp,forward_records,cancel_records,email,inbound_email,inbound_phonecall,inperson_contact,internal_note,left_message,outbound_email,outbound_phonecall,phonecall,text,unknown,gender,childrenPresentInHH,maritalStatus,lengthOfResidenceInYrs,annualHHIncome,education,urbanicity,credits_after_refund,is_Lockdown,NumberofGamesPerSeason,CNTPostponedGames,isNextYear_Buyer
0,28,355875895,18,2016,Partial Plan,310.08,2016-08-20,0.823529,164,TRUE,304,304,19.61,14,2,1,1,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,74,None,0
1,28,355876102,202,2016,Partial Plan,220.08,2016-05-17,0.250000,119,TRUE,255,255,14.99,3,8,1,0,1,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,74,None,1
2,28,355876729,255,2016,Partial Plan,620.16,2016-08-22,0.470588,87,TRUE,227,227,160.03,10,1,2,0,2,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,74,None,0
3,28,355877001,165,2016,Partial Plan,110.04,2016-05-23,0.250000,158,TRUE,298,298,10.56,3,8,1,0,1,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,74,None,0
4,28,355877239,191,2016,Partial Plan,110.04,2016-05-23,0.250000,135,TRUE,275,275,2.00,3,8,1,0,1,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,74,None,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2014 entries, 0 to 2013
Data columns (total 54 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   lkupClientId            2014 non-null   int64  
 1   dimCustomerMasterId     2014 non-null   int64  
 2   customerNumber          2014 non-null   object 
 3   year                    2014 non-null   int64  
 4   productGrouping         2014 non-null   object 
 5   totalSpent              2014 non-null   float64
 6   recentDate              2014 non-null   object 
 7   attendancePercent       2014 non-null   float64
 8   renewedBeforeDays       2014 non-null   int64  
 9   isBuyer                 2014 non-null   object 
 10  source_tenure           2014 non-null   int64  
 11  tenure                  2014 non-null   int64  
 12  distToVenue             2014 non-null   float64
 13  totalGames              2014 non-null   int64  
 14  recency                 2014 non-null   

### Let's also drop the features that only have a single value, as they won't add much differentiation to our model:

In [5]:
for col in df.columns:
    if len(df[col].unique()) == 1:
        df.drop(col,inplace=True,axis=1)
        
df.shape

(2014, 20)

### We should also drop features that have a low correlation with the target label as they won't be useful for prediction, we'll only keep features that have a correlation above a set threshold:

In [6]:
cor = df.corr()

threshold = 0.05

#Correlation with output variable
cor_target = abs(cor["isNextYear_Buyer"])

#Selecting highly correlated features
relevant_features = cor_target[cor_target > threshold]

feats = []
for name, val in relevant_features.items():
    feats.append(name)

df_correlated = df[feats]

df_correlated.shape

df_correlated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2014 entries, 0 to 2013
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   dimCustomerMasterId     2014 non-null   int64  
 1   year                    2014 non-null   int64  
 2   totalSpent              2014 non-null   float64
 3   renewedBeforeDays       2014 non-null   int64  
 4   distToVenue             2014 non-null   float64
 5   totalGames              2014 non-null   int64  
 6   missed_games_1          2014 non-null   int64  
 7   missed_games_2          2014 non-null   int64  
 8   missed_games_over_2     2014 non-null   int64  
 9   is_Lockdown             2014 non-null   int64  
 10  NumberofGamesPerSeason  2014 non-null   int64  
 11  isNextYear_Buyer        2014 non-null   int64  
dtypes: float64(2), int64(10)
memory usage: 188.9 KB


### Now that we have the right features we can look at the correlations between them, if features are highly correlated with each other it might negatively impact the model:

In [7]:
corr = df_correlated.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

,dimCustomerMasterId,year,totalSpent,renewedBeforeDays,distToVenue,totalGames,missed_games_1,missed_games_2,missed_games_over_2,is_Lockdown,NumberofGamesPerSeason,isNextYear_Buyer
dimCustomerMasterId,1.00,0.15,0.32,0.06,0.10,0.16,0.04,0.06,0.22,0.07,0.11,0.12
year,0.15,1.00,-0.09,-0.15,0.06,-0.14,-0.10,-0.08,-0.10,0.78,0.80,-0.19
totalSpent,0.32,-0.09,1.00,0.26,0.01,0.60,0.16,0.25,0.64,-0.20,-0.09,0.30
renewedBeforeDays,0.06,-0.15,0.26,1.00,-0.04,0.28,0.15,0.19,0.27,-0.25,-0.18,0.27
distToVenue,0.10,0.06,0.01,-0.04,1.00,-0.03,-0.02,-0.02,-0.01,0.05,0.05,-0.05
totalGames,0.16,-0.14,0.60,0.28,-0.03,1.00,0.62,0.57,0.59,-0.21,-0.14,0.37
missed_games_1,0.04,-0.10,0.16,0.15,-0.02,0.62,1.00,0.52,0.23,-0.10,-0.11,0.23
missed_games_2,0.06,-0.08,0.25,0.19,-0.02,0.57,0.52,1.00,0.36,-0.12,-0.09,0.23
missed_games_over_2,0.22,-0.10,0.64,0.27,-0.01,0.59,0.23,0.36,1.00,-0.22,-0.11,0.29
is_Lockdown,0.07,0.78,-0.20,-0.25,0.05,-0.21,-0.10,-0.12,-0.22,1.00,0.50,-0.30


### In order to compare two sets of features, we need to create some datasets for training and evalution:


In [8]:
# select % of the data for training
df_train = df_correlated.sample(frac=0.8, random_state=786).reset_index(drop=True)

# create the eval datasets for A and B
df_eval = df_correlated.drop(df_train.index).reset_index(drop=True)

# print out the number of records for training and eval
print('Data for Modeling: ' + str(df_train.shape))
print('Unseen Data For Predictions: ' + str(df_eval.shape), end="\n\n")

Data for Modeling: (1611, 12)
Unseen Data For Predictions: (403, 12)



### Now we can model the data using a binary classification prediction for the isnextyear_buyer field to see how likely a customer is to re-purchase.

In [9]:
setup(
    data= df_train, 
    target="isNextYear_Buyer", 
    train_size = 0.80,
    data_split_shuffle=True,
    silent=True,
    numeric_features=[
        "totalSpent",
        "renewedBeforeDays",
        "distToVenue",
        "totalGames",
        "missed_games_1",
        "missed_games_2",
        "missed_games_over_2",
        "is_Lockdown",
        "NumberofGamesPerSeason"
    ]
)

,Description,Value
0,session_id,1473
1,Target,isNextYear_Buyer
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(1611, 12)"
5,Missing Values,False
6,Numeric Features,10
7,Categorical Features,1
8,Ordinal Features,False
9,High Cardinality Features,False


(10,
 5,
       dimCustomerMasterId   totalSpent  renewedBeforeDays  distToVenue  \
 0             355881952.0   330.119995              160.0    12.900000   
 1             356037792.0     0.000000              150.0     7.030000   
 2             443943072.0  3558.800049              140.0    12.900000   
 3             356039136.0     0.000000              300.0    12.210000   
 4             356037120.0   220.000000                1.0    18.389999   
 ...                   ...          ...                ...          ...   
 1606          356051744.0   315.000000               49.0    49.759998   
 1607          356071328.0  1899.800049              148.0     2.000000   
 1608          356076032.0  2380.000000               76.0    30.469999   
 1609          355880192.0   220.000000              171.0    14.990000   
 1610          392804256.0     0.000000               27.0     8.020000   
 
       totalGames  missed_games_1  missed_games_2  missed_games_over_2  \
 0            7

In [10]:
model_matrix = compare_models(
    fold=10,
    include=["lr", "xgboost"]
)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.7236,0.7994,0.7512,0.7150,0.7319,0.4471,0.4486,0.2840
lr,Logistic Regression,0.5023,0.5590,1.0000,0.5023,0.6687,0.0000,0.0000,0.3570


In [11]:
best_model = create_model(model_matrix)
final_model = finalize_model(best_model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7442,0.7704,0.8000,0.7222,0.7591,0.4879,0.4908
1,0.7132,0.8180,0.7846,0.6892,0.7338,0.4257,0.4299
2,0.6977,0.7933,0.6923,0.7031,0.6977,0.3954,0.3954
3,0.7442,0.8296,0.7077,0.7667,0.7360,0.4886,0.4901
4,0.6899,0.7536,0.7231,0.6812,0.7015,0.3795,0.3802
5,0.6434,0.7332,0.6923,0.6338,0.6618,0.2863,0.2875
6,0.7519,0.8233,0.8154,0.7260,0.7681,0.5034,0.5073
7,0.7597,0.8356,0.7812,0.7463,0.7634,0.5195,0.5201
8,0.7422,0.8130,0.7656,0.7313,0.7481,0.4844,0.4849
9,0.7500,0.8237,0.7500,0.7500,0.7500,0.5000,0.5000


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

## Observations
Here you can document some ideas on the results from above

## Conclusions
Here you can talk about next steps, did the experiment work? If yes, what to do next? If no, why?